# Hosting Capacity Deterministic Analysis

In [9]:
import os, math, sys
import pandas as pd
import pandapower as pp
import numpy as np

from pandapower.file_io import from_json, to_json

main_dir = os.path.abspath(os.path.join(os.path.abspath(""), ".."))
data_dir = os.path.join(main_dir, 'Modified_116_LV_CSV')

if main_dir not in sys.path:
    sys.path.insert(0, main_dir)

import create_basic_network as cbn
import pp_simu

In [10]:
net = from_json(os.path.join(main_dir, 'json_networks', 'full_load_network.json'))
print(net)
net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].describe().to_csv(os.path.join(data_dir, 'full_net.res_bus_3ph_describe.csv'))

profile_df, ds = cbn.create_data_source(data_dir)
ds.df.head(10)

This pandapower network includes the following parameter tables:
   - bus (117 elements)
   - asymmetric_load (55 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (115 elements)
   - trafo (1 element)
 and the following results tables:
   - res_bus_3ph (117 elements)
   - res_line_3ph (115 elements)
   - res_trafo_3ph (1 element)
   - res_ext_grid_3ph (1 element)
   - res_shunt_3ph (1 element)
   - res_asymmetric_load_3ph (55 elements)


,LOAD1,LOAD1_Q,LOAD2,LOAD2_Q,LOAD3,LOAD3_Q,LOAD4,LOAD4_Q,LOAD5,LOAD5_Q,...,LOAD51,LOAD51_Q,LOAD52,LOAD52_Q,LOAD53,LOAD53_Q,LOAD54,LOAD54_Q,LOAD55,LOAD55_Q
time,,,,,,,,,,,,,,,,,,,,,
00:01:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000048,0.000016,0.000035,0.000012,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000048,0.000016,0.000055,0.000018
00:02:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000048,0.000016,0.000035,0.000012,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000048,0.000016,0.000055,0.000018
00:03:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000048,0.000016,0.000035,0.000012,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000057,0.000019,0.000055,0.000018
00:04:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000227,0.000075,0.000035,0.000012,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000048,0.000016,0.000055,0.000018
00:05:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000227,0.000075,0.000035,0.000012,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000048,0.000016,0.000055,0.000018
00:06:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000227,0.000075,0.000035,0.000012,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000048,0.000016,0.000055,0.000018
00:07:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000227,0.000075,0.000147,0.000048,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000048,0.000016,0.000055,0.000018
00:08:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000227,0.000075,0.000147,0.000048,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000148,0.000049,0.000055,0.000018
00:09:00,0.000036,0.000012,0.000044,0.000014,0.000054,0.000018,0.000227,0.000075,0.000147,0.000048,...,0.000053,0.000017,0.000052,0.000017,0.000047,0.000015,0.000148,0.000049,0.000055,0.000018


In [11]:
output_path = os.path.join(main_dir, 'hc_results', 'DET')
os.makedirs(output_path) if not os.path.exists(output_path) else None

### 1. PV-only scenario

In [12]:
pv_hc_results = pp_simu.hc_deterministic(net, add_kw=1.0, max_kw=20.0, pv=True, ev=False)

pv_hc_results.to_csv(os.path.join(output_path, 'pv_hc_results.csv'))

1 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 0.0 kW
2 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 1.0 kW
3 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 2.0 kW
4 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 3.0 kW
5 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 4.0 kW
6 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 5.0 kW
7 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 6.0 kW
8 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 7.0 kW
9 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 8.0 kW
10 PV Gen created so far...
Overvoltage violation at bus 2, phase A with 9.0 kW
11 PV Gen created so far...
Stopped at bus 2, phase A with 10.0 kW due to error: Power Flow nr did not converge after 100 iterations!
1 PV Gen created so far...
Overvoltage violation at bus 2, phase B with 0.0

### 2. EV-only scenario

In [13]:
ev_hc_results = pp_simu.hc_deterministic(net, add_kw=1.0, max_kw=20.0, pv=False, ev=True)

ev_hc_results.to_csv(os.path.join(output_path, 'ev_hc_results.csv'))

1 EVs created so far...
Overvoltage violation at bus 2, phase A with 0.0 kW
2 EVs created so far...
Overvoltage violation at bus 2, phase A with 1.0 kW
3 EVs created so far...
Overvoltage violation at bus 2, phase A with 2.0 kW
4 EVs created so far...
Stopped at bus 2, phase A with 3.0 kW due to error: Power Flow nr did not converge after 100 iterations!
1 EVs created so far...
Overvoltage violation at bus 2, phase B with 0.0 kW
2 EVs created so far...
Stopped at bus 2, phase B with 1.0 kW due to error: Power Flow nr did not converge after 100 iterations!
1 EVs created so far...
Overvoltage violation at bus 2, phase C with 0.0 kW
2 EVs created so far...
Overvoltage violation at bus 2, phase C with 1.0 kW
3 EVs created so far...
Overvoltage violation at bus 2, phase C with 2.0 kW
4 EVs created so far...
Overvoltage violation at bus 2, phase C with 3.0 kW
5 EVs created so far...
Stopped at bus 2, phase C with 4.0 kW due to error: Power Flow nr did not converge after 100 iterations!
1 EVs

### 3. PV+EV scenario

In [14]:
pvev_hc_results = pp_simu.hc_deterministic(net, add_kw=1.0, max_kw=20.0, pv=True, ev=True)

pvev_hc_results.to_csv(os.path.join(output_path, 'pvev_hc_results.csv'))

1 PV Gen created so far...
1 EVs created so far...
Overvoltage violation at bus 2, phase A with 0.0 kW
2 PV Gen created so far...
2 EVs created so far...
Overvoltage violation at bus 2, phase A with 1.0 kW
3 PV Gen created so far...
3 EVs created so far...
Overvoltage violation at bus 2, phase A with 2.0 kW
4 PV Gen created so far...
4 EVs created so far...
Overvoltage violation at bus 2, phase A with 3.0 kW
5 PV Gen created so far...
5 EVs created so far...
Overvoltage violation at bus 2, phase A with 4.0 kW
6 PV Gen created so far...
6 EVs created so far...
Overvoltage violation at bus 2, phase A with 5.0 kW
7 PV Gen created so far...
7 EVs created so far...
Overvoltage violation at bus 2, phase A with 6.0 kW
8 PV Gen created so far...
8 EVs created so far...
Overvoltage violation at bus 2, phase A with 7.0 kW
9 PV Gen created so far...
9 EVs created so far...
Overvoltage violation at bus 2, phase A with 8.0 kW
10 PV Gen created so far...
10 EVs created so far...
Overvoltage violatio